[View in Colaboratory](https://colab.research.google.com/github/isaacyaf/example/blob/master/titanic_survival_prediction_DNN_lite.ipynb)

# Predict Survival on the Titanic


![ref. kaggle](https://kaggle2.blob.core.windows.net/competitions/kaggle/3136/logos/header.png)

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.

ref. kaggle-Titanic

In [2]:

# get titanic & test csv files
!curl -L -o titanic_train.csv "https://drive.google.com/uc?export=download&id=1Ao0T6DKiCweVleaGt4rOw625gf7qmzou"
!curl -L -o titanic_test.csv "https://drive.google.com/uc?export=download&id=1EgVFol5j7LI8gweqiQiZ3HWO3S6O7vGT"

  
# pandas, numpy, matplotlib, sns, missingno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re as re

#keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from IPython.display import clear_output

#set the jupyter notebook
%matplotlib inline

plt.style.use('ggplot')
def plot_train_history(history, train_metrics, val_metrics):
    plt.plot(history.history.get(train_metrics),'-o')
    plt.plot(history.history.get(val_metrics),'-o')
    plt.ylabel(train_metrics)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
    
def find_data(df,col,data):
  pd_tmp = pd.DataFrame(columns=df.columns.values)
  if data == None:
    pd_tmp = df[df[col].isna()] 
  else:
    for index, data_tmp in enumerate(df[col]):
      if data in str(data_tmp):
        pd_tmp=pd.concat([pd_tmp,df[index:index+1]])
  return pd_tmp

clear_output()

# get titanic data as a DataFrame
titanic_df = pd.read_csv('titanic_train.csv')
test_df    = pd.read_csv('titanic_test.csv')

full_data = [titanic_df, test_df]

# preview the data
titanic_df.head()

,Unnamed: 0,PassengerId,Pclass,Name,Name_length,Title,Sex,Age,Age_Categories,SibSp,Parch,FamilySize,IsAlone,Ticket,Fare,Fare_Categories,Cabin,Has_Cabin,Embarked,Survived
0,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51,3,0,38.0,2,1,0,2,0,PC 17599,71.2833,3,C85,1,1,1
1,2,3,3,"Heikkinen, Miss. Laina",22,2,0,26.0,1,0,0,1,1,STON/O2. 3101282,7.9250,1,NaN,0,0,1
2,3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",44,3,0,35.0,2,1,0,2,0,113803,53.1000,3,C123,1,0,1
3,4,5,3,"Allen, Mr. William Henry",24,1,1,35.0,2,0,0,1,1,373450,8.0500,1,NaN,0,0,0
4,5,6,3,"Moran, Mr. James",16,1,1,22.0,1,0,0,1,1,330877,8.4583,1,NaN,0,2,0



|Col. Name|Col. Explanation| Data Explanation |
|---	|---	|--- |
|PassengerId|通行證號碼||   
|Pclass         |社會經濟地位 |1/2/3  = 一等/二等/三等 |
|Name           |乘客姓名||
|Name_length           |乘客姓名長度||
|Title           |頭銜| 1: Mr <br> 2: Miss   <br> 3: Mrs <br> 4: Master  <br> 5: Rare <br><br> Rare = 'Capt', 'Col', 'Countess',	'Don', 'Dr',<br> 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir'|
|Sex            |性別|0: Female <br> 1: Male|
|Age            |年齡||
|Age            |年齡分類|0: ≤ 16 <br> 1: > 16 &  ≤ 32  <br> 2: > 32 &  ≤ 48  <br> 3: > 48 & ≤ 64  <br> 4: > 64 |
|SibSp          |手足或配偶也在船上的數量||
|Parch          |父母子女也在船上的數量||
|FamilySize          |整個家族在船上的數量|SibSp + Parch + 1|
|IsAlone          |是否跟家族一起搭船|0: 否 <br> 1: 是|
|Ticket         |船票資訊||
|Fare           |票價 (單位: 英鎊)|
|Fare_Categories |票價分類|0: ≤ 7.91 <br> 1: > 7.91 &  ≤ 14.454  <br> 2: > 14.454 &  ≤ 31  <br> 3: > 31  |
|Cabin          |客艙資訊||
|Has_Cabin          |是否有記載客艙資訊|0: 否 <br> 1: 是|
|Embarked       |登船港口 | <br> 0 = Southampton (英格蘭-南安普敦) <br> 1 = Cherbourg(法國-瑟堡)  <br> 2 = Queenstown (紐西蘭-皇后鎮)  |
|Survived       |是否存活 |0 = 否, 1 =是|


# Feature List & Model Building
'Pclass', 'Name_length', 'Title', 'Sex', 'Age', 'Age_Categories', 'SibSp', 'Parch', 'FamilySize', 'IsAlone', 'Fare', 'Fare_Categories', 'Has_Cabin', 'Embarked'

* Pick features that you feel are important

```
# selected_list = ['SibSp', 'Parch', 'FamilySize']
```




In [3]:
#'Pclass', 'Name_length', 'Title', 'Sex', 'Age', 'Age_Categories', 'SibSp', 
#'Parch', 'FamilySize', 'IsAlone', 'Fare', 'Fare_Categories', 'Has_Cabin', 'Embarked'

#================ Select your features ========================
selected_list = ['SibSp', 'Parch', 'FamilySize']

#============================================================


train_tmp = titanic_df[selected_list].values
train_DLy = titanic_df[['Survived']].values
inputdims = len(train_tmp[0])
batch_size = 10
nb_classes = 1
nb_epoch = 30


#================ Build your model ========================
model = Sequential()
model.add(Dense(16, input_dim=inputdims, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#============================================================


model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


#Learning
history = model.fit(train_tmp, 
                    train_DLy,
                    batch_size=batch_size, 
                    epochs=nb_epoch,
                    verbose=2, 
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________
Train on 793 samples, validate on 89 samples
Epoch 1/30
 - 2s - loss: 0.6998 - acc: 0.5095 - val_loss: 0.6764 - val_acc: 0.6404
Epoch 2/30
 - 1s - loss: 0.6747 - acc: 0.6116 - val_loss: 0.6567 - val_acc: 0.6404
Epoch 3/30
 - 1s - loss: 0.6647 - acc: 0.6166 - val_loss: 0.6

In [22]:
nametmp = 'Duane, Mr. Frank'

print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))

#print(display(find_data(test_df, 'Name', nametmp)[selected_list]))
display(find_data(test_df, 'Name', nametmp)[selected_list])

#with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#    print(find_data(test_df, 'Name', nametmp)[selected_list])

display(test_df[0:5][selected_list])



[[0.3024969]]


,SibSp,Parch,FamilySize
0,0,0,1


,SibSp,Parch,FamilySize
0,0,0,1
1,0,0,1
2,1,0,2
3,1,2,4
4,0,0,1


In [3]:


nametmp = 'Duane, Mr. Frank'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Svensson, Mr. Johan'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Braund, Mr. Owen Harris'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Herman, Miss. Alice'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Turkula, Mrs. (Hedwig)'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))


nametmp = 'Guggenheim, Mr. Benjamin'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Byles, Rev. Thomas Roussel Davids'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Rosalie Ida Blun'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Lines, Miss. Mary Conover'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))
nametmp = 'Blank, Mr. Henry'
print(model.predict(find_data(test_df, 'Name', nametmp)[selected_list].values))

[[1]]
[]
[[0.30957973]]
[[0.30957973]]
[[0.5078436]]
[[0.50693697]]
[[0.30957973]]
[[0.30957973]]
[[0.30957973]]
[[0.5078436]]
[[0.6093185]]
[[0.30957973]]


In [0]:
JandR_df = pd.DataFrame(
  [ 
  [3, "Jack Dawson", 11, 1, 1, 20.0, 1, 0, 0, 1, 1, "x", 7.0, 0, "x", 0, 0, 0],
  [1, "Rose DeWitt Bukater", 19, 2, 0, 17.0, 1, 0, 0, 1, 1, "x", 20.0, 2, "x", 1, 0, 1]
  ]
, columns=['Pclass', 'Name', 'Name_length', 'Title', 'Sex', 'Age', 
           'Age_Categories', 'SibSp', 'Parch', 'FamilySize', 'IsAlone', 
           'Ticket', 'Fare', 'Fare_Categories', 'Cabin', 'Has_Cabin', 
           'Embarked','Survived'])

#JandR_df

print(model.predict_classes(find_data(JandR_df, 'Name', 'Jack')[selected_list].values))
print(model.predict_classes(find_data(JandR_df, 'Name', 'Rose')[selected_list].values))
    

,Pclass,Name,Name_length,Title,Sex,Age,Age_Categories,SibSp,Parch,FamilySize,IsAlone,Ticket,Fare,Fare_Categories,Cabin,Has_Cabin,Embarked,Survived
0,3,Jack Dawson,11,1,1,20.0,1,0,0,1,1,x,7.0,0,x,0,0,0
1,1,Rose DeWitt Bukater,19,2,0,17.0,1,0,0,1,1,x,20.0,2,x,1,0,1


In [0]:
#'Pclass', 'Name_length', 'Title', 'Sex', 'Age', 'Age_Categories', 'SibSp', 
#'Parch', 'FamilySize', 'IsAlone', 'Fare', 'Fare_Categories', 'Has_Cabin', 'Embarked'

#titanic_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False).plot.bar()
titanic_df[["Sex", "Survived"]].groupby(['Sex']).mean().sort_values(by='Survived', ascending=False).plot.bar()
titanic_df[["Title", "Survived"]].groupby(['Title']).mean().plot.bar()
titanic_df[["Age", "Survived"]].groupby(['Age']).mean().plot.bar()
titanic_df[["Pclass", "Survived"]].groupby(['Pclass']).mean().plot.bar()
titanic_df[["FamilySize", "Survived"]].groupby(['FamilySize']).mean().plot.bar()

titanic_df[["Embarked", "Survived"]].groupby(['Embarked']).mean().plot.bar()
#titanic_df[["Embarked", "Fare_Categories"]].groupby(['Fare_Categories']).plot.bar()

titanic_df.groupby(['Embarked', 'Fare_Categories']).count()[['PassengerId']]

In [0]:
titanic_df[["Age_Categories", "Survived"]].groupby(['Age_Categories']).mean().plot.bar()
titanic_df[["Fare_Categories", "Survived"]].groupby(['Fare_Categories']).mean().plot.bar()

In [0]:
Y_pred = model.predict_classes(test_df[selected_list])
Y_pred = map(lambda x: x[0], Y_pred)
Y_pred = pd.Series(Y_pred)

In [0]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_tmp, train_DLy)
X_test=test_df[selected_list].values.reshape(-1, len(test_df[selected_list].loc[0]))
Y_pred = random_forest.predict(X_test)
#print(random_forest.score)

In [0]:
print(random_forest.score(train_tmp, train_DLy))
print (Y_pred)
X_test=test_df[selected_list]
print(X_test.loc[0:2].values.reshape(-1, len(X_test.loc[0])))


In [0]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

In [0]:
submission.to_csv('submission.csv', index=False)

from google.colab import files
#uploaded = files.upload() # then browse, select the files. It's then uploaded

files.download('submission.csv')

In [0]:
titanic_df.groupby(['Pclass', 'Has_Cabin']).count()[['PassengerId']]

PassengerId
Pclass Has_Cabin             
1      0                   40
       1                  176
2      0                  168
       1                   16
3      0                  479
       1                   12

In [0]:
#find_data(titanic_df, 'Pclass', '1')
titanic_df[(titanic_df.Survived == 1)  & (titanic_df.Pclass == 1) ]

,Unnamed: 0,PassengerId,Pclass,Name,Name_length,Title,Sex,Age,Age_Categories,SibSp,Parch,FamilySize,IsAlone,Ticket,Fare,Fare_Categories,Cabin,Has_Cabin,Embarked,Survived
2,2,3,3,"Heikkinen, Miss. Laina",22,2,0,26.00,1,0,0,1,1,STON/O2. 3101282,7.9250,1,NaN,0,0,1
8,8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",49,3,0,27.00,1,0,2,3,0,347742,11.1333,1,NaN,0,0,1
10,10,11,3,"Sandstrom, Miss. Marguerite Rut",31,2,0,4.00,0,1,1,3,0,PP 9549,16.7000,2,G6,1,0,1
19,19,20,3,"Masselmani, Mrs. Fatima",23,3,0,22.00,1,0,0,1,1,2649,7.2250,0,NaN,0,1,1
22,22,23,3,"McGowan, Miss. Anna ""Annie""",27,2,0,15.00,0,0,0,1,1,330923,8.0292,1,NaN,0,2,1
25,25,26,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",57,3,0,38.00,2,1,5,7,0,347077,31.3875,3,NaN,0,0,1
28,28,29,3,"O'Dwyer, Miss. Ellen ""Nellie""",29,2,0,22.00,1,0,0,1,1,330959,7.8792,0,NaN,0,2,1
32,32,33,3,"Glynn, Miss. Mary Agatha",24,2,0,22.00,1,0,0,1,1,335677,7.7500,0,NaN,0,2,1
36,36,37,3,"Mamee, Mr. Hanna",16,1,1,22.00,1,0,0,1,1,2677,7.2292,0,NaN,0,1,1
39,39,40,3,"Nicola-Yarred, Miss. Jamila",27,2,0,14.00,0,1,0,2,0,2651,11.2417,1,NaN,0,1,1
